In [ ]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def fetus_detection(image, pixel_size):
    
    image_with_contour = image.copy()
    
    #step 1
    # filter the image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_blur = cv2.medianBlur(gray,3)
    #img_blur = gray
    
    #step 2
    # enhance the image
    hist= cv2.equalizeHist(img_blur)
    # filter again
    img_blur = cv2.medianBlur(hist,25)
    # threshold
    threshed_image = cv2.adaptiveThreshold(img_blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    # filter again
    img_blur = cv2.medianBlur(threshed_image,3)
    
    #step 3
    # edge detection
    canny_edges = cv2.Canny(img_blur,0,300)
    
    #step 4
    # Apply hough transform on the image
    circles = cv2.HoughCircles(canny_edges, cv2.HOUGH_GRADIENT, 1, image.shape[0]/0.5, param1=200, param2=10, minRadius=70, maxRadius=130)
    radius = circles[0][0][2]
    circumference = 2*np.pi*radius*pixel_size
    
    #step 5
    # Draw detected circles
    our_circle = np.zeros((image.shape[0],image.shape[1],3))
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            # Draw the circle on original image
            cv2.circle(image_with_contour, (i[0], i[1]), i[2], (0, 255, 0), 2)
            # draw the circle on black screen
            cv2.circle(our_circle, (i[0], i[1]), i[2], (0, 255, 0), 2)
            
    our_circle_clipped = our_circle.astype(np.uint8)
    
    return [canny_edges, image_with_contour, our_circle_clipped, circumference]

In [ ]:
def calculate_metrics(circumference, original_circumference, annote_image, our_circle):
    
    #metric 1
    # Difference
    difference = circumference - original_circumference
    
    #metric 2
    # Absolute Difference
    absolute_difference = np.abs(circumference - original_circumference)
    
    #metric 3
    # Dice Coeffecient
    def dice(im1, im2):

        im1 = np.asarray(im1).astype(np.bool)
        im2 = np.asarray(im2).astype(np.bool)

        if im1.shape != im2.shape:
            raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

        # Compute Dice coefficient
        intersection = np.logical_and(im1, im2)

        return 2. * intersection.sum() / (im1.sum() + im2.sum())
    
    # shade the annote circle
    editable_image = annote_image.copy()
    gray = cv2.cvtColor(annote_image, cv2.COLOR_BGR2GRAY)
    contour, _ = cv2.findContours(gray,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    annote_contour = cv2.drawContours(editable_image, contour, -1, (0,255,0), 1)
    cv2.fillPoly(annote_contour, pts =[contour[0]], color=(255,255,255))
    
    # shade our circle got from hough transform
    our_gray_circle = cv2.cvtColor(our_circle, cv2.COLOR_BGR2GRAY)
    contour, _ = cv2.findContours(our_gray_circle,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cv2.fillPoly(our_circle, pts =[contour[0]], color=(255,255,255))
    
    dice_coeffecient = dice(annote_contour, our_circle)
    
    return [difference, absolute_difference, dice_coeffecient]

In [ ]:
def plot_images(original, edges, detected_image, our_circle, annote_image):
    
    figure, (ax1,ax2,ax3,ax4,ax5) = plt.subplots(1,5, sharex=True, sharey=True, gridspec_kw={'hspace': 0, 'wspace': 0}, figsize = (15,15))
    
    original = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
    detected_image = cv2.cvtColor(detected_image, cv2.COLOR_BGR2RGB)
    annote_image = cv2.cvtColor(annote_image, cv2.COLOR_BGR2RGB)
    our_circle = cv2.cvtColor(our_circle, cv2.COLOR_BGR2RGB)
    
    ax1.imshow(original)
    ax2.imshow(edges, cmap='gray')
    ax3.imshow(detected_image)
    ax4.imshow(our_circle)
    ax5.imshow(annote_image)
    
    ax1.set_title('original')
    ax2.set_title('detected_edges')
    ax3.set_title('fetus_detection')
    ax4.set_title('my_fetus_detection')
    ax5.set_title('actual_fetus_detection')

In [ ]:
image = cv2.imread('29/114_HC.png')
annote_image = cv2.imread('29/114_HC_Annotation.png')
pixel_size = 0.131386392995
original_circumference = 82.33
[canny_edges, image_with_contour, our_circle, circumference] = fetus_detection(image, pixel_size)
plot_images(image, canny_edges, image_with_contour, our_circle, annote_image)
metrics = calculate_metrics(circumference, original_circumference, annote_image, our_circle)
print('Scores')
print('Difference = ', metrics[0],"mm")
print('Absolute Difference = ', metrics[1],"mm")
print('Dice Coeffecient = ', metrics[2])

In [ ]:
image = cv2.imread('29/115_HC.png')
annote_image = cv2.imread('29/115_HC_Annotation.png')
pixel_size = 0.107903929167
original_circumference = 77.52
[canny_edges, image_with_contour, our_circle, circumference] = fetus_detection(image, pixel_size)
plot_images(image, canny_edges, image_with_contour, our_circle, annote_image)
metrics = calculate_metrics(circumference, original_circumference, annote_image, our_circle)
print('Scores')
print('Difference = ', metrics[0],"mm")
print('Absolute Difference = ', metrics[1],"mm")
print('Dice Coeffecient = ', metrics[2])

In [ ]:
image = cv2.imread('29/116_HC.png')
annote_image = cv2.imread('29/116_HC_Annotation.png')
pixel_size = 0.104077743987
original_circumference = 82.05
[canny_edges, image_with_contour, our_circle, circumference] = fetus_detection(image, pixel_size)
plot_images(image, canny_edges, image_with_contour, our_circle, annote_image)
metrics = calculate_metrics(circumference, original_circumference, annote_image, our_circle)
print('Scores')
print('Difference = ', metrics[0],"mm")
print('Absolute Difference = ', metrics[1],"mm")
print('Dice Coeffecient = ', metrics[2])

In [ ]:
image = cv2.imread('29/117_HC.png')
annote_image = cv2.imread('29/117_HC_Annotation.png')
pixel_size = 0.109634995461
original_circumference = 80.88
[canny_edges, image_with_contour, our_circle, circumference] = fetus_detection(image, pixel_size)
plot_images(image, canny_edges, image_with_contour, our_circle, annote_image)
metrics = calculate_metrics(circumference, original_circumference, annote_image, our_circle)
print('Scores')
print('Difference = ', metrics[0],"mm")
print('Absolute Difference = ', metrics[1],"mm")
print('Dice Coeffecient = ', metrics[2])

In [ ]:
image = cv2.imread('29/118_HC.png')
annote_image = cv2.imread('29/118_HC_Annotation.png')
pixel_size = 0.101576994876
original_circumference = 76.6
[canny_edges, image_with_contour, our_circle, circumference] = fetus_detection(image, pixel_size)
plot_images(image, canny_edges, image_with_contour, our_circle, annote_image)
metrics = calculate_metrics(circumference, original_circumference, annote_image, our_circle)
print('Scores')
print('Difference = ', metrics[0],"mm")
print('Absolute Difference = ', metrics[1],"mm")
print('Dice Coeffecient = ', metrics[2])